Reads a parquet file, repartitions it, and writes partitions to parquet files.
Gets the file size of a single parquet file (hardcoded)

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [12]:
data_dir = "./data/yellow_tripdata_2024-10.parquet"
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -O $data_dir # download oct 2024 data from the official source

--2025-03-08 19:54:48--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.140, 54.230.209.200, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘./data/yellow_tripdata_2024-10.parquet’

./data/yellow_tripd 100%[===================>]  61.36M   159MB/s    in 0.4s    

2025-03-08 19:54:49 (159 MB/s) - ‘./data/yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]

254534 ./data/yellow_tripdata_2024-10.parquet


In [13]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/08 19:55:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
# read dataframe and show schema
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .parquet(data_dir)
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [19]:
num_p = 4
df = df.repartition(num_p)

In [20]:
parquet_dir = "./data/parquet/"
df.write.parquet(parquet_dir)

In [25]:
import os
parquet_fpath = "./data/parquet/part-00000-09836a4e-cd38-4e5f-aa2f-82cb68c978c6-c000.snappy.parquet"
print(f"{os.path.getsize(parquet_fpath) / 1024 / 1024: .2f} MB")

 24.88 MB
